In [1]:
%load_ext autoreload
%autoreload 2

import csv
import numpy as np
import os
import pandas as pd
import time

from benchmarks import gsa_svm_fitness
from src.entities import GSA

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from typing import Mapping, Tuple, Union
from ucimlrepo import fetch_ucirepo

In [2]:
# Fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# Data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
X = X.fillna(value=0)  # Fill missing values with 0

y = breast_cancer_wisconsin_original.data.targets 

In [3]:
class UCI:
    """
    Class to handle UCI datasets
    
    Args:
        X (pd.DataFrame): Features
        y (pd.Series): Target
    """
    def __init__(self,
                 X: pd.DataFrame,
                 y: pd.Series
                 ) -> None:
        """
        Constructor
        
        Args:
            X (pd.DataFrame): Features
            y (pd.Series): Target
        """
        self.X = X
        self.y = y
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        min_train = X_train.min()
        range_train = (X_train - min_train).max()
        self.X_train_scaled = (X_train - min_train)/range_train
        
        min_test = X_test.min()
        range_test = (X_test - min_test).max()
        self.X_test_scaled = (X_test - min_test)/range_test
    
    def get_fitness(self,
                    solution: Mapping[str, np.ndarray],
                    data: Union[None, Tuple[np.ndarray, np.ndarray]] = None,
                    ) -> Tuple[float, float]:
        """
        Get fitness of a solution
        
        Args:
            solution (Mapping[str, np.ndarray]): Solution to evaluate
            data (Union[None, Tuple[np.ndarray, np.ndarray]], optional): Data to evaluate the solution. Defaults to None.
        
        Returns:
            Tuple[float, float]: Fitness and accuracy of the solution    
        """
        if data is None:
            X_scaled = self.X_train_scaled
            y_data = self.y_train
        else:
            X_scaled, y_data = data
            
        C, gamma = solution['real']
        X_scaled_filtered = X_scaled.iloc[:, solution['discrete'].astype(int) == 1]
        svc_model = SVC(C=C, gamma=gamma, kernel="rbf", verbose=False)
        svc_model.fit(X_scaled_filtered, np.ravel(y_data))
        y_predict = svc_model.predict(X_scaled_filtered)
        conf_matrix = confusion_matrix(y_data, y_predict)
        accuracy = accuracy_score(y_data, y_predict) * 100
        
        return gsa_svm_fitness(accuracy=accuracy, solution=solution)

uci = UCI(X, y)

In [4]:
solution = {'real': [5, 20], 'discrete': np.array([0, 1, 1, 0, 0, 1, 0, 0, 1])}
uci.get_fitness(solution=solution)

(78.96620950109323, 98.56887298747765)

In [20]:
# Select number of repetitions for each experiment. 
runs = 10

# Select chaotic constant
chaotic_constant = True

# Export results ?
export = True

save_path = "data/output/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

ExportToFile = save_path + "experiment" + time.strftime("%Y-%m-%d-%H-%M-%S_") + str(chaotic_constant) + ".csv"

# Check if it works at least once
atLeastOneIteration = False

population_size = 5 
iterations = 20

fitness_evaluations = 100

boundaries = {'real': [(0.0001, 100), (0.0001, 100)], 'discrete': [(0, 1) for _ in range(len(X.columns))]}

# CSV Header for the convergence
CnvgHeader = ["Sol_Iter"+str(l+1) for l in range(iterations)]

best_solution_history = []
accuracy_history = []
gamma_history = []
c_history = []
n_features_history = []

uci = UCI(X, y)

for k in range(runs):
    gsa_algo = GSA(objective_function = uci.get_fitness,
                   r_dim=2,
                   d_dim=len(X.columns),
                   boundaries=boundaries)
    
    gsa_algo.optimize(population_size=population_size,
                      iters=iterations,
                      chaotic_constant=False)
    
    print(gsa_algo.solution_history[-1])
    fitness, accuracy = uci.get_fitness(solution=gsa_algo.solution_history[-1],
                                        data=(uci.X_test_scaled, uci.y_test))
    
    print("Test accuracy: ", accuracy, " - Fitness: ", fitness)
    
    best_solution_history.append(gsa_algo.solution_history[-1])
    accuracy_history.append(accuracy)
    gamma_history.append(gsa_algo.solution_history[-1]['real'][1])
    c_history.append(gsa_algo.solution_history[-1]['real'][0])
    n_features_history.append(np.sum(gsa_algo.solution_history[-1]['discrete']))
    
    if export:
        with open(ExportToFile, 'a') as out:
            writer = csv.writer(out, delimiter=',')
            if not atLeastOneIteration:  # just one time to write the header of the CSV file
                header = np.concatenate(
                    [["Optimizer", "objfname", "startTime", "EndTime", "ExecutionTime"], CnvgHeader])
                writer.writerow(header)
            a = np.concatenate(
                [[gsa_algo.objective_function_name,
                  gsa_algo.start_time,
                  gsa_algo.end_time,
                  gsa_algo.execution_time],
                  gsa_algo.convergence])
            writer.writerow(a)
        out.close()
    atLeastOneIteration = True  # at least one experiment

if not atLeastOneIteration:  # Failed to run at least one experiment
    print("No Optimizer or Cost function is selected. Check lists of available optimizers and cost functions")

GSA is optimizing  "get_fitness"
['At iteration 1 the best fitness is 78.39375869608429']
['At iteration 2 the best fitness is 78.63553965414431']
['At iteration 3 the best fitness is 78.63553965414431']
['At iteration 4 the best fitness is 78.63553965414431']
['At iteration 5 the best fitness is 78.63553965414431']
['At iteration 6 the best fitness is 78.63553965414431']
['At iteration 7 the best fitness is 78.63553965414431']
['At iteration 8 the best fitness is 78.63553965414431']
['At iteration 9 the best fitness is 78.63553965414431']
['At iteration 10 the best fitness is 78.63553965414431']
['At iteration 11 the best fitness is 78.63553965414431']
['At iteration 12 the best fitness is 78.63553965414431']
['At iteration 13 the best fitness is 78.63553965414431']
['At iteration 14 the best fitness is 78.63553965414431']
['At iteration 15 the best fitness is 78.63553965414431']
['At iteration 16 the best fitness is 78.63553965414431']
['At iteration 17 the best fitness is 78.6355396

In [21]:
best_solution_history

[{'real': array([3.31113056, 1.95673869]),
  'discrete': array([1, 1, 0, 1, 0, 1, 0, 1, 1])},
 {'real': array([ 1.38924708, 12.49937893]),
  'discrete': array([0, 1, 0, 1, 0, 0, 1, 1, 0])},
 {'real': array([18.38694104,  0.04451101]),
  'discrete': array([1, 1, 1, 1, 1, 1, 0, 1, 1])},
 {'real': array([5.02254001, 1.77143844]),
  'discrete': array([0, 1, 0, 1, 1, 1, 0, 1, 0])},
 {'real': array([0.07481162, 0.55451193]),
  'discrete': array([1, 1, 0, 1, 0, 1, 0, 0, 1])},
 {'real': array([11.36895282, 19.56601299]),
  'discrete': array([1, 0, 0, 0, 1, 1, 1, 1, 1])},
 {'real': array([2.38539228e+01, 2.35525341e-02]),
  'discrete': array([1, 1, 1, 1, 0, 1, 1, 0, 1])},
 {'real': array([ 6.04598705, 17.53484964]),
  'discrete': array([1, 0, 1, 0, 1, 0, 1, 1, 0])},
 {'real': array([ 1.94935657, 13.41250784]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 0, 0])},
 {'real': array([ 1.36528758, 17.00352315]),
  'discrete': array([0, 1, 1, 1, 0, 0, 1, 1, 1])}]

In [22]:
accuracy_history

[98.57142857142858,
 96.42857142857143,
 93.57142857142857,
 96.42857142857143,
 94.28571428571428,
 100.0,
 93.57142857142857,
 99.28571428571429,
 100.0,
 99.28571428571429]

In [23]:
mean_accuracy = np.mean(accuracy_history)
std_accuracy = np.std(accuracy_history)

print(f"Mean accuracy: {mean_accuracy} +/- {std_accuracy}")

mean_gamma = np.mean(gamma_history)
std_gamma = np.std(gamma_history)

print(f"Mean gamma: {mean_gamma} +/- {std_gamma}")

mean_c = np.mean(c_history)
std_c = np.std(c_history)

print(f"Mean C: {mean_c} +/- {std_c}")

mean_n_features = np.mean(n_features_history)
std_n_features = np.std(n_features_history)

print(f"Mean n_features: {mean_n_features} +/- {std_n_features}")

Mean accuracy: 97.14285714285714 +/- 2.4948927423675715
Mean gamma: 8.436702514578514 +/- 7.816948269088931
Mean C: 7.276817713966489 +/- 7.6626667720125194
Mean n_features: 5.9 +/- 1.1357816691600546


In [19]:
solution= {'real': np.array([0.91833448, 34.69824839]),
           'discrete': np.array([1, 1, 0, 1, 1, 1, 0, 1, 0])}

fitness, accuracy = uci.get_fitness(solution=solution,
                                    data=(uci.X_test_scaled, uci.y_test))

print("Test accuracy: ", accuracy, " - Fitness: ", fitness)

Test accuracy:  100.0  - Fitness:  80.06666666666666
